In [ ]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# Japanese IQ Test

The following puzzle is part of a recruitment test in Japan.

---

A policeman, a convict, a father and his two sons Anton and Bruno, and a mother with her two daughters Cindy and Doris have to cross a river.  On the boat there is only room for two passengers.

During the crossing, the following conditions have to be observed:
* The father is not allowed to be on a shore with one of the daughters 
  if the mother is on the other shore. 
* The mother is not allowed to be on a shore with one of the sons 
  if the father is on the other shore.
* If the criminal is not alone, then the policeman must watch him.

  However the convict can be alone on a shore, as his shackles prevent him from running away. 
* Only the father, the mother, and the policeman are able to steer the boat. 

We are going to formulate this problem as a *symbolic transition problem*.
Then, we can solve the problem using our constraint solver.

In [ ]:
import z3

We will use the following variables to encode the problem:
* `P` equals `1` if the policeman is on the western shore,
* `V` equals `1` if the villain is on the western shore,
* `M` equals `1` if the mother is on the western shore,
* `F` equals `1` if the father is on the western shore,
* `B` is the number of boys on the western shore,
* `G` is the number of girls on the western shore.

In [ ]:
def start(P, V, M, F, B, G):
    "your code here"

In [ ]:
def goal(P, V, M, F, B, G):
    "your code here"

In [ ]:
def invariant(P, V, M, F, B, G):
    "your code here"

The last parameter `i` of the function `transistion` specifies the number of the crossing.

In [ ]:
def transition(P𝛼, V𝛼, M𝛼, F𝛼, B𝛼, G𝛼,
               P𝛽, V𝛽, M𝛽, F𝛽, B𝛽, G𝛽,
               i
              ):
    if i % 2 == 0:
        "your code here"
    else:
        "your code here"
    return Formulas

In [ ]:
def japanese_CSP(n):
    S = z3.Solver()
    """
    your code here
    """
    result = str(S.check())
    if result == 'sat':
        Model = S.model()
        Solution = (   { f'P{i}': Model[Ps[i]] for i in range(n+1) }
                     | { f'V{i}': Model[Vs[i]] for i in range(n+1) }
                     | { f'M{i}': Model[Ms[i]] for i in range(n+1) }
                     | { f'F{i}': Model[Fs[i]] for i in range(n+1) }
                     | { f'B{i}': Model[Bs[i]] for i in range(n+1) }
                     | { f'G{i}': Model[Gs[i]] for i in range(n+1) }
                   )
        return { key: Solution[key].as_long() for key in Solution }
    else:
        return None

In [ ]:
def find_solution():
    n = 1
    while True:
        print(n)
        Solution = japanese_CSP(n)
        if Solution != None:
            return n, Solution
        n += 2

In [ ]:
%%time
n, Solution = find_solution()
n, Solution

## Auxiliary Code for Pretty Printing

The following code is used for printing the path that has been found.  We won't discuss the details of these functions.

In [ ]:
def show_solution(Solution, n):
    for i in range(n+1):
        P = Solution[f'P{i}']
        V = Solution[f'V{i}']
        M = Solution[f'M{i}']
        F = Solution[f'F{i}']
        B = Solution[f'B{i}']
        G = Solution[f'G{i}']
        print('👮'*P+'🦹'*V+'👩'*M+'👨🏿'*F+'👦'*B+'👧🏾'*G + ' '*42 + \
              '👮'*(1-P)+'🦹'*(1-V)+'👩'*(1-M)+'👨🏿'*(1-F)+'👦'*(2-B)+'👧🏾'*(2-G))
        if i % 2 == 0:
            PS = Solution[f'P{i}'] - Solution[f'P{i+1}']
            VS = Solution[f'V{i}'] - Solution[f'V{i+1}']
            MS = Solution[f'M{i}'] - Solution[f'M{i+1}']
            FS = Solution[f'F{i}'] - Solution[f'F{i+1}']
            BS = Solution[f'B{i}'] - Solution[f'B{i+1}']
            GS = Solution[f'G{i}'] - Solution[f'G{i+1}']
            print(' '*24+'>>>'+'👮'*PS+'🦹'*VS+'👩'*MS+'👨🏿'*FS+'👦'*BS+'👧🏾'*GS+'>>>')
        elif i + 1 < n:
            PS = Solution[f'P{i+1}'] - Solution[f'P{i}']
            VS = Solution[f'V{i+1}'] - Solution[f'V{i}']
            MS = Solution[f'M{i+1}'] - Solution[f'M{i}']
            FS = Solution[f'F{i+1}'] - Solution[f'F{i}']
            BS = Solution[f'B{i+1}'] - Solution[f'B{i}']
            GS = Solution[f'G{i+1}'] - Solution[f'G{i}']
            print(' '*24+'<<<'+'👮'*PS+'🦹'*VS+'👩'*MS+'👨🏿'*FS+'👦'*BS+'👧🏾'*GS+'<<<')

In [ ]:
show_solution(Solution, n)